# Analytic Preparation: 

In [38]:
%%capture
!pip install splunk-sdk
!pip install pandas
!pip install pandasql
import matplotlib.pyplot as plt
import pandas as pd
import pandasql
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 0)

In [39]:
import splunklib.client as client
# Create a Service instance and log in 
service = client.connect(
    host='192.168.1.68', #Input your Splunk's IP here
    port='8089',
    username="admin",
    password="Changeme1!",
    scheme='https')

# Title: Service Creation (Detection)
####  Author: Jonathan Johnson & Jared Atkinson

## Goal: 
To alert when a new value is made within the `Services` registry key. 

## Categorization:

#### [Windows Service Creation](https://attack.mitre.org/techniques/T1543/003/)

## Tooling: 
#### [Sysmon](https://docs.microsoft.com/en-us/sysinternals/downloads/sysmon#:~:text=System%20Monitor%20(Sysmon)%20is%20a,changes%20to%20file%20creation%20time.)

## Base Condition: 
Registry key creation under the `HKLM\SYSTEM\CurrentControlSet\Services` key

## Base Context Attributes: 
- Service Name Created
- Image that created the service
- ProcessGUID of the creating process
- Timestamp

## Base Condition Analytic: 

In [40]:
import splunklib.results as results
#Query: 
query = "search index=windows sourcetype=Sysmon EventCode=12 earliest=-15m | regex TargetObject=\"^HKLM\\\\\System\\\\\CurrentControlSet\\\\\Services\\\\\[a-zA-Z0-9_\\-]*$\""
query_results = service.jobs.oneshot(query, count=0)
reader = results.ResultsReader(query_results)

results = []

for result in reader:
    results.append(result)
    
df_EID_12=pd.DataFrame(results)
df2_EID_12 = df_EID_12.Message
df3_EID_12 = df2_EID_12.str.split('\n')
df4_EID_12 = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_12]
df5_EID_12 = pd.DataFrame(df4_EID_12)

In [42]:
from pandasql import sqldf
Base_Condition = pandasql.sqldf(
"""
SELECT *
FROM df5_EID_12
WHERE EventType = "CreateKey"
"""

).drop_duplicates().reset_index().drop("index", axis=1)

In [43]:
%store Base_Condition

Stored 'Base_Condition' (DataFrame)


In [37]:
Base_Condition.to_pickle('~/work/Base_Condition')